In [1]:
from pyquil import Program
from pyquil.gates import *
import pyquil.api as api
from pyquil.paulis import *
import numpy as np
import math
import random

In [2]:
from grove.pyvqe.vqe import VQE
from scipy.optimize import minimize
import numpy as np

# Setting up connection to qvm and VQE
vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
qvm = api.QVMConnection()

In [8]:
def U(n):
    """
    Builds the interaction term: a_p^dagger a_q^dagger a_q a_p
    """
    U = 2
    
    hamiltonian = ZERO()
    for i in range(n):
        hamiltonian += (sI() - sZ(i) - sZ(i+n) + sZ(i)*sZ(i+n))
    hamiltonian *= (U / 4)
        
    return hamiltonian

def t():
    """
    Builds the hopping term: a_p^dagger a_q
    Currently only for N = 4
    """
    t = 1
    
    hamiltonian = ZERO()
    hamiltonian = (sX(0)*sX(1) + sY(0)*sY(1) + sX(0)*sZ(1)*sZ(2)*sX(3) + sY(0)*sZ(1)*sZ(2)*sY(3) + sX(1)*sX(2) + sY(1)*sY(2) + sX(2)*sX(3) + sY(2)*sY(3))
    hamiltonian += (sX(4)*sX(5) + sY(4)*sY(5) + sX(4)*sZ(5)*sZ(6)*sX(7) + sY(4)*sZ(5)*sZ(6)*sY(7) + sX(5)*sX(6) + sY(5)*sY(6) + sX(6)*sX(7) + sY(6)*sY(7))
    hamiltonian *= (-t / 2)
    
    return hamiltonian

hamU = U(4)
hamt = t()

hamiltonian = hamU + hamt
print(hamiltonian)

(2+0j)*I + (-0.5+0j)*Z0 + (-0.5+0j)*Z4 + (0.5+0j)*Z0*Z4 + (-0.5+0j)*Z1 + (-0.5+0j)*Z5 + (0.5+0j)*Z1*Z5 + (-0.5+0j)*Z2 + (-0.5+0j)*Z6 + (0.5+0j)*Z2*Z6 + (-0.5+0j)*Z3 + (-0.5+0j)*Z7 + (0.5+0j)*Z3*Z7 + (-0.5+0j)*X0*X1 + (-0.5+0j)*Y0*Y1 + (-0.5+0j)*X0*Z1*Z2*X3 + (-0.5+0j)*Y0*Z1*Z2*Y3 + (-0.5+0j)*X1*X2 + (-0.5+0j)*Y1*Y2 + (-0.5+0j)*X2*X3 + (-0.5+0j)*Y2*Y3 + (-0.5+0j)*X4*X5 + (-0.5+0j)*Y4*Y5 + (-0.5+0j)*X4*Z5*Z6*X7 + (-0.5+0j)*Y4*Z5*Z6*Y7 + (-0.5+0j)*X5*X6 + (-0.5+0j)*Y5*Y6 + (-0.5+0j)*X6*X7 + (-0.5+0j)*Y6*Y7


In [9]:
from qucochemistry.utils import pyquilpauli_to_qubitop
from openfermion.transforms import get_sparse_operator

# Use exact diagonalization on the qubit hamiltonian to get the ground state energy
# It is sometimes not the lowest energy in the list below because we are restricted to the half filling sector
h = get_sparse_operator(pyquilpauli_to_qubitop(hamiltonian))
if h.shape[0] > 1024:
    [w, _] = sp.sparse.linalg.eigsh(h, k=1)
else:
    [w, _] = np.linalg.eigh(h.todense())
w[0:10]

array([-3.62721301, -3.20925146, -3.20925146, -3.20925146, -3.20925146,
       -2.82842712, -2.68584617, -2.68584617, -2.68584617, -2.        ])

In [10]:
from pyquil.quil import DefGate
from pyquil.quilatom import Parameter, quil_sin, quil_cos

# Defining a controlled-RY gate
theta = Parameter('theta')
cry = np.array([[1,0,0,0],
               [0,1,0,0],
               [0,0,quil_cos(theta), -quil_sin(theta)],
               [0,0,quil_sin(theta), quil_cos(theta)]])

cry_definition = DefGate("CRY", cry, [theta])
CRY = cry_definition.get_constructor()

In [11]:
def U_ansatz(n, param):
    """
    Creates a circuit for e^{i \theta_U H_U}
    """
    p = Program()
    
    for i in range(n):
        p.inst(RZ(param/4, i))
        p.inst(RZ(param/4, i+n))
        p.inst(CNOT(i, i+n))
        p.inst(RZ(-param/4, i+n))
        p.inst(CNOT(i, i+n))
        
    return p

In [12]:
def t_ansatz(s1, s2, param):
    """
    Creates a circuit for e^{i \theta_t H_t} for horizontal and vertical hoppings
    """
    p = Program()
    
    for i in range(s1+1, s2):
        if (i == s2-1):
            p.inst(CZ(i, i+1))
        else:
            p.inst(CNOT(i, i+1))
    
    p.inst(H(s1))
    p.inst(H(s2))
    p.inst(CNOT(s1, s2))
    p.inst(RZ(param, s2))
    p.inst(CNOT(s1, s2))
    p.inst(H(s1))
    p.inst(H(s2))
    p.inst(RX(-np.pi/2, s1))
    p.inst(RX(-np.pi/2, s2))
    p.inst(CNOT(s1, s2))
    p.inst(RZ(param, s2))
    p.inst(CNOT(s1, s2))
    p.inst(RX(-np.pi/2, s1))
    p.inst(RX(-np.pi/2, s2))  
    
    for i in reversed(range(s1+1, s2)):
        if (i == s2-1):
            p.inst(CZ(i, i+1))
        else:
            p.inst(CNOT(i, i+1))
    
    return p

In [13]:
from openfermion.utils import slater_determinant_preparation_circuit

# Q matrix for ground state
arr = np.array([[0.5,0.5,0.5,0.5],
                [0.5,-0.5,-0.5,0.5]])    

def slater_determinant(): 
    """
    Prepares the Slater determinant for U = 0 as described in https://arxiv.org/pdf/1711.05395.pdf
    """
    givens = slater_determinant_preparation_circuit(arr)

    def givens_rotation(tups, spin):
        p = Program()

        for tup in tups:
            # where tup is (j, k, theta, phi)
            p.inst(CNOT(tup[1]+4*spin, tup[0]+4*spin))

            # controlled-RY
            p.inst(CRY(tup[2])(tup[0]+4*spin, tup[1]+4*spin))

            p.inst(CNOT(tup[1]+4*spin, tup[0]+4*spin))
            p.inst(RZ(tup[3], tup[1]+4*spin))
        
        return p
    
    p = Program()
    p += cry_definition
    
    # Fill first N_f orbitals for each spin
    p.inst(X(0))
    p.inst(X(1))
    p.inst(X(4))
    p.inst(X(5))

    for rot in givens:
        p += givens_rotation(rot, 0)
        p += givens_rotation(rot, 1)
        
    return p
print(slater_determinant_preparation_circuit(arr))
#print(slater_determinant())

[((1, 2, 0.7853981633974484, 0.0),), ((0, 1, -1.5707963267948966, 0.0), (2, 3, -1.5707963267948966, 0.0)), ((1, 2, -0.7853981633974484, 0.0),)]


In [14]:
def var_ansatz(params):
    """
    Prepares \sum_S [U_U(\theta_U / 2) U_h(\theta_h) U_v(\theta_v) U_U(\theta_U / 2)] | \Psi_I >
    Where | \Psi_I > is the Slater determinant for U = 0
    """
    S = 3
    p = Program()
    
    p += slater_determinant()
    
    for i in range(S):
        # building U_u
        p += U_ansatz(4, params[3*i])
        
        # building U_t, horizontal and vertical
        p += t_ansatz(0, 1, params[3*i+1])
        p += t_ansatz(2, 3, params[3*i+1])
        p += t_ansatz(4, 5, params[3*i+1])
        p += t_ansatz(6, 7, params[3*i+1])

        p += t_ansatz(0, 3, params[3*i+2])
        p += t_ansatz(1, 2, params[3*i+2])
        p += t_ansatz(4, 7, params[3*i+2])
        p += t_ansatz(5, 6, params[3*i+2])
        
        # building U_u
        p += U_ansatz(4, params[3*i])
        
    return p

In [17]:
#initial angle
#initial_params = np.random.rand(9)
#result = vqe_inst.vqe_run(var_ansatz, hamiltonian, initial_params, None, qvm=qvm)
#result

In [35]:
from scipy.optimize import minimize

def expectation(x):
    """
    Expectation value for parameters, x
    """
    return vqe_inst.expectation(var_ansatz(x), hamiltonian, None, qvm)

def scipy_minimize(x):
    """
    Minimizes expectation value using parameters found in greedy search as starting point
    Powell's conjugate direction method
    """
    return minimize(expectation, x, method='powell')

def greedy_noisy_search(initial_state):
    """
    Slightly perturb the values of the points, accepting whenever this results in a lower energy
    Total of 150 evaulations of the energy
    Change step size after 30 evaluations based on number of acceptances in previous trial group
    """
    params = initial_state
    min_energy = vqe_inst.expectation(var_ansatz(initial_state), hamiltonian, None, qvm)

    def random_point(dim, step):
        """
        Generates a random point on the perimeter of a circle with radius, step
        """
        coords = [random.gauss(0, 1) for i in range(dim)]
        norm = math.sqrt(sum([i**2 for i in coords]))
        coords = [(i / norm) * step for i in coords]
        return coords

    step = 0.1
    acceptances = 0

    # Five groups of 30 trials
    for i in range(5):
        acceptances = 0
        for j in range(30):
            
            # Slightly perturb the values of the points
            coords = random_point(len(initial_state), step)
            temp_params = [sum(x) for x in zip(params, coords)]
            
            # Calculate expectation value with new parameters
            temp_energy = vqe_inst.expectation(var_ansatz(temp_params), hamiltonian, None, qvm)
            
            # Greedily accept parameters that result in lower energy
            if (temp_energy < min_energy):
                min_energy = temp_energy
                params = temp_params
                acceptances += 1
        # Update step size
        step *= (acceptances / 15)

    return {'x': params, 'energy': min_energy, 'step': step}

def global_variational(initial_params=[]):
    """
    Complete a round of greedy and Powell's optimization for six randomly chosen points
    Further optimizes the best point
    """
    points = []
    #points = [np.random.rand(9)*0.2 for i in range(6)]
    if (len(initial_params) != 0):
        points.append(initial_params)
    greedy_results = [greedy_noisy_search(x) for x in points]
    results = [scipy_minimize(i['x']) for i in greedy_results]
    res = min(results, key=lambda x:x.fun)
    print(res)
    # For the chosen point, we continue optimizing until we cannot find improvement
    res_copy = res
    energy = res_copy.fun
    while (True):
        result = greedy_noisy_search(res_copy.x)
        temp_res = scipy_minimize(result['x'])
        temp_energy = temp_res.fun
        print(temp_energy)

        # Optimizing will usually find an energy ~1e-6 lower
        # Stop it eventually
        tolerance = 0.00001
        if (abs(temp_energy - energy) > tolerance):
            res_copy = temp_res
            energy = temp_energy
        else:
            break
    return res_copy

In [38]:
res = global_variational([ 3.14159265e+00, -1.09277732e-01, -1.36639333e-01,  3.14523799e+00,
        4.53254180e-05,  5.12117453e-01,  3.13402122e+00,  2.73691117e-04,
        5.12074733e-01])
print(res)

   direc: array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]])
     fun: array(-2.5720382)
 message: 'Optimization terminated successfully.'
    nfev: 123
     nit: 1
  status: 0
 success: True
       x: array([ 3.14179397e+00, -1.09464781e-01, -1.36574405e-01,  3.14845529e+00,
        8.75092569e-05,  5.12108900e-01,  3.13402122e+00,  2.79369408e-04,
        5.12075736e-01])
-2.5720382606931294
   direc: array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0

In [ ]:
def op(s1, s2):
    """
    returns a PauliSum representing a_s1^daggar a_s2
    """
    def sigma_plus(s):
        """
        returns sigma^+ operator on side s
        sigma^+ = 1/2(X + iY)
        """
        return PauliTerm('X', s, 0.5) + PauliTerm('Y', s, complex(0.0+0.5j))
    def sigma_minus(s):
        """
        returns sigma^- operator on side s
        sigma^- = 1/2(X - iY)
        """
        return PauliTerm('X', s, 0.5) - PauliTerm('Y', s, complex(0.0+0.5j))
    
    # for all sites in between site one (s1) and site two (s2), multiply by sigma z
    z = sI()
    for i in range(s1+1, s2):
        z *= PauliTerm('Z', i)
        
    return sigma_minus(s1) * z * sigma_plus(s2)

In [ ]:
# building the single particle density matrix (spdm) 
spdm = np.zeros((4,4))
for i in range(4):
    for k in range(4):
        # building an operator and vqe for each i, j        
        # element (j, i) in the spdm is a_i^dagger a_j
        spdm[k, i] = vqe_inst.expectation(slater_determinant(), op(i, k), None, qvm)
        
print(np.around(spdm, decimals=2))